## COMP 790.175 Computational Imaging and Displays - Assignment 3: ISP
### EMRE ONEMLI

#### import necessary libraries

In [3]:
import numpy as np

### 1.1 Implement a basic image processing pipeline (80 points)

dcraw: This is the command name, which invokes the dcraw program.

-4: This flag tells dcraw to output the image in 16-bit linear or "linear gamma" format. By default, dcraw processes images in 8-bit. Using 16-bit allows for more precision, which is beneficial for high-quality post-processing, especially in shadows and highlights.

-d: This option forces dcraw to use the document mode. In document mode, dcraw skips the color interpolation step, resulting in an image where each pixel corresponds directly to a single sensor element. This mode is sometimes referred to as producing a "RAW" image, but in a more direct sense, since no demosaicing is applied. It's useful for archival purposes or when maximum detail is required without interpolation artifacts.

-v: This flag makes dcraw operate in verbose mode, where it outputs detailed information about the processing steps to the console. This can include information about the camera, the image, and the specific processing actions dcraw is taking.

-w: This option tells dcraw to use the camera's white balance settings as specified in the RAW file. Without this flag, dcraw might use its own calculations or defaults for white balance, which could differ from what the camera's software determined.

-T: This flag instructs dcraw to output a TIFF file instead of the default PPM format. TIFF files are widely used in professional photography and graphic design because they support higher bit depths and can include metadata. This is useful for further processing or editing in other software.

In [10]:
!dcraw -4 -d -v -w -T ../data/baby.nef

shell-init: error retrieving current directory: getcwd: cannot access parent directories: No such file or directory
Loading Nikon D3 image from ../data/baby.nef ...
Scaling with darkness 0, saturation 16383, and
multipliers 1.628906 1.000000 1.386719 1.000000
Building histograms...
Writing data to ../data/baby.tiff ...


In [15]:
'''
Loading Nikon D3 image from ../data/baby.nef ...
Scaling with darkness 0, saturation 16383, and
multipliers 1.628906 1.000000 1.386719 1.000000
Building histograms...
Writing data to ../data/baby.tiff ...
'''
''

''